In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import config as APIConfig
from copy import deepcopy
import pandas as pd

In [3]:
# API = 'https://on-ramp.api.cx.metamask.io/providers'


# query_params = {
#     "regionId":"/regions/de"
#     , "paymentMethodId":"/payments/debit-credit-card"
#     , "cryptoCurrencyId": "/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7"
#     , "fiatCurrencyId": "/currencies/fiat/eur"
#     , "amount":95
# }

# # headers = {
# #     "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
# #     , "Origin": "https://portfolio.metamask.io"
# #     , "Referer": "https://portfolio.metamask.io/"
# #     , "Content-Type": "application/json; charset=utf-8"
# #     , 
# # }
# response = requests.get(API)


# Scrape Payment data for regions

In [10]:
REGIONS = [
    {
        "region":"de"
        , "country":"DE"
        , "fiat_currency": "/currencies/fiat/eur"
    },
    {
        "region":"us-fl"
        , "country":"US"
        , "fiat_currency": "/currencies/fiat/usd"
    },
    {
        "region":"gb"
        , "country":"UK"
        , "fiat_currency": "/currencies/fiat/gb"
    }
]

REGION_PARAMS = {
    "sdk":"1.28.3"
}


REGIONS_API = 'https://on-ramp-content.api.cx.metamask.io/regions'

In [4]:
def extract_payments_data(payment_dict:dict):
    payment_dict.pop('logo', None)
    payment_dict.pop('icons', None)
    return payment_dict

def add_region_payment_data(region_dict:str, region_api:str = REGIONS_API, query_params:dict = REGION_PARAMS):
    # using a deepcopy to avoid mutating the original input
    dict_copy = deepcopy(region_dict)
    response = requests.get(f"{region_api}/{dict_copy['region']}", params=query_params)
    try:
        data = response.json()
        payment_info = [extract_payments_data(x) for x in data["payments"]]
        dict_copy["payments"] = payment_info
        print("Add Payment data")
        return dict_copy
    except Exception as e:
        print("Error adding payment data")
        print(e)

In [11]:
REGIONS_WITH_PAYMENT = []
for region in REGIONS:
    print(f"extracting payment info for", region)
    region_it = add_region_payment_data(region)
    REGIONS_WITH_PAYMENT.append(region_it)    

extracting payment info for {'region': 'de', 'country': 'DE', 'fiat_currency': '/currencies/fiat/eur'}
Add Payment data
extracting payment info for {'region': 'us-fl', 'country': 'US', 'fiat_currency': '/currencies/fiat/usd'}
Add Payment data
extracting payment info for {'region': 'gb', 'country': 'UK', 'fiat_currency': '/currencies/fiat/gb'}
Add Payment data


# Generate Quote Scrapings

In [14]:
REGIONS_WITH_PAYMENT[0]

{'region': 'de',
 'country': 'DE',
 'fiat_currency': '/currencies/fiat/eur',
 'payments': [{'id': '/payments/apple-pay',
   'paymentType': 'apple-pay',
   'name': 'Apple Pay',
   'score': 60,
   'disclaimer': 'Apple Cash is not supported.',
   'delay': [0, 0],
   'pendingOrderDescription': 'Card purchases may take a few minutes to complete.',
   'amountTier': [1, 3],
   'isApplePay': True,
   'sell': {'enabled': False},
   'recurringBuy': {'enabled': False},
   'buy': {'enabled': True}},
  {'id': '/payments/debit-credit-card',
   'paymentType': 'debit-credit-card',
   'name': 'Debit or Credit',
   'score': 90,
   'disclaimer': "Credit card purchases may incur your bank's cash advance fees, subject to your bank's policies.",
   'delay': [5, 10],
   'pendingOrderDescription': 'Card purchases may take a few minutes to complete.',
   'amountTier': [1, 3],
   'sellEnabled': True,
   'sell': {'enabled': True},
   'recurringBuy': {'enabled': True},
   'buy': {'enabled': True}},
  {'id': '/pay

In [26]:
CRYPTOCURRENCIES = [
    {
        "name": "Ethereum Mainnet"
        , "id": "/currencies/crypto/1/0x0000000000000000000000000000000000000000"
    },
    {
        "name": "USDT (Ethereum)"
        , "id":  "/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7"
    },
    {
        "name": "USDT (BNB Chain)"
        , "id":  "/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955"
    },
    
]

QUOTE_QUERY_PARAMS_BASE = []
for region in REGIONS_WITH_PAYMENT:
    for payment in region["payments"]:
        for crypto in CRYPTOCURRENCIES:
            QUOTE_QUERY_PARAMS_BASE.append({
                "country": region["country"]
                , "region": region["region"] 
                , "crypto_name": crypto["name"]
                , "payment_name": payment["name"]
                , "scrape_params": {
                    "regionId":f"/regions/{region['region']}"
                    , "paymentMethodId": payment["id"]
                    , "cryptoCurrencyId": crypto["id"]
                    , "fiatCurrencyId": region["fiat_currency"]
                }
                
            })

In [27]:
test_quote = deepcopy(QUOTE_QUERY_PARAMS_BASE[0])
test_quote["scrape_params"]["amount"] = 50

In [53]:
QUOTE_QUERY_PARAMS_BASE[20]

{'country': 'US',
 'region': 'us-fl',
 'crypto_name': 'USDT (BNB Chain)',
 'payment_name': 'Debit or Credit',
 'scrape_params': {'regionId': '/regions/us-fl',
  'paymentMethodId': '/payments/debit-credit-card',
  'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
  'fiatCurrencyId': '/currencies/fiat/usd'}}

In [71]:
QUOTE_API = 'https://on-ramp.api.cx.metamask.io/providers/all/quote'


# r = requests.get(QUOTE_API, params=test_quote["scrape_params"])


In [49]:
def _get_quote_data(quote_json:dict):
    mandatory_keys = ["amountIn", "amountOut", "exchangeRate", "networkFee", "providerFee"]
    optional_fields = [{"key":"bestRate", "default":False}]
    quote_clean_dict = {key:val for key,val in quote_json.items() if key in mandatory_keys}
    for field in optional_fields:
        quote_clean_dict[field["key"]] = quote_json.get(field["key"], field["default"])
    return quote_clean_dict

def get_quote_data(provider_quote_json, _position:int):
    quote_data = _get_quote_data(provider_quote_json["quote"])
    quote_data.update({
        "provider": provider_quote_json["provider"]
        , "position": _position
    })
    return quote_data
    

In [70]:
QUOTE_API

'https://on-ramp.api.cx.metamask.io'

In [83]:
def scrape_one_quote(scrape_params: dict , amount: float, endpoint=QUOTE_API):
    copy_params = deepcopy(scrape_params)
    copy_params["amount"] = amount
    response = requests.get(endpoint, params=copy_params)
    print(response.status_code)
    # return response
    try:
        raw = response.json()["success"]
        clean_quotes = []
        for i,provider_quote in enumerate(raw):
            _ = get_quote_data(provider_quote, i)
            _.update(copy_params)
            clean_quotes.append(_)
        return {"status": "success", "payload": clean_quotes}
    except Exception as e: 
        print("Error scraping", copy_params)
        print(e)
        return {"status": "failure", "payload": response}

In [ ]:
    # try:
    #     raw = response.json()["success"]
    #     clean_quote = [get_quote_data(provider_quote, i) for i,provider_quote in enumerate(raw)]
    #     clean_quote.update(copy_params)
    # except Exception as e: 
    #     print("Error scraping", copy_params)
    #     print(e)

In [82]:
r = scrape_one_quote(scrape_params=QUOTE_QUERY_PARAMS_BASE[20]["scrape_params"], amount = 100, endpoint=QUOTE_API)

200
Error scraping {'regionId': '/regions/us-fl', 'paymentMethodId': '/payments/debit-credit-card', 'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955', 'fiatCurrencyId': '/currencies/fiat/usd', 'amount': 100}
'list' object has no attribute 'update'


In [78]:
r.json()

{'success': [{'provider': '/providers/banxa',
   'url': None,
   'method': None,
   'headers': None,
   'quote': {'crypto': {'id': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
     'idv2': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
     'legacyId': '/currencies/crypto/56/usdt',
     'network': {'active': True,
      'chainId': '56',
      'chainName': 'BNB Chain Mainnet',
      'shortName': 'BNB Chain'},
     'logo': 'https://tokens.1inch.io/0xdac17f958d2ee523a2206206994597c13d831ec7.png',
     'decimals': 18,
     'address': '0x55d398326f99059ff775485246999027b3197955',
     'symbol': 'USDT',
     'name': 'Tether USD'},
    'cryptoId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
    'fiat': {'id': '/currencies/fiat/usd',
     'symbol': 'USD',
     'name': 'US Dollar',
     'decimals': 2,
     'denomSymbol': '$'},
    'fiatId': '/currencies/fiat/usd',
    'amountIn': 100,
    'amountOut': 99.97,
    'exchangeRate':

In [72]:
r.url

'https://on-ramp.api.cx.metamask.io/?country=US&region=us-fl&crypto_name=USDT+%28BNB+Chain%29&payment_name=Debit+or+Credit&scrape_params=regionId&scrape_params=paymentMethodId&scrape_params=cryptoCurrencyId&scrape_params=fiatCurrencyId&amount=100'

In [69]:
r.json()

{'product': 'On-Ramp',
 'service': 'On-Ramp API',
 'version': '1.40.1',
 'features': ['orders', 'analytics']}

In [66]:
QUOTE_QUERY_PARAMS_BASE[20]

{'country': 'US',
 'region': 'us-fl',
 'crypto_name': 'USDT (BNB Chain)',
 'payment_name': 'Debit or Credit',
 'scrape_params': {'regionId': '/regions/us-fl',
  'paymentMethodId': '/payments/debit-credit-card',
  'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
  'fiatCurrencyId': '/currencies/fiat/usd'}}

In [64]:

r.json()



{'product': 'On-Ramp',
 'service': 'On-Ramp API',
 'version': '1.40.1',
 'features': ['orders', 'analytics']}